In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing as scale
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from sklearn import mixture
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

We do the repeat experiment

In [27]:
input_file = ["Backdoor.csv" , "DDoS_HTTP.csv" , "DDoS_ICMP.csv" , "DDoS_TCP.csv" , "DDoS_UDP.csv"  , "Password.csv" , "Port_Scanning.csv" ,"Ransomware.csv", "SQL_injection.csv" , "Uploading.csv" , "Vulnerability_scanner.csv" ,"XSS.csv" ]

In [28]:
def pred(x, clfs, p_nodes, gms):
    p_x_node = np.zeros((len(x), len(clfs))) # p[x, in node i] = p(x | x in Node i) * p(x in Node i)
    p_y_given_node = np.zeros((len(x), len(clfs))) # p[y | x, node i]
    for i in range(len(clfs)):
        p_x_node[:, i] = p_nodes[i] * np.exp(gms[i].score_samples(x))
        p_y_given_node[:, i] = clfs[i].predict_proba(x)[:, 1]
    
    p_y = p_y_given_node * p_x_node / (np.sum(p_x_node, axis=-1, keepdims=True) + 1e-10)

    p_y = np.hstack([1-np.sum(p_y, axis=-1, keepdims=True), p_y])

    return np.argmax(p_y, axis=-1)

In [29]:
columns_to_drop_bi = ['Attack_type']

data_list = []
for i in range(len(input_file)):
    data = pd.read_csv(input_file[i])
    data['attack_type'] = i + 1
    data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
    data.drop(columns = columns_to_drop_bi, inplace = True)
    if len(data) < 2000:
        data_list.append(data)
    else:
        data0 = data[data['attack_type'].isin([0])]
        x1 = data0.sample(n = 1000, random_state = i + 42, axis = 0)
        data1 = data[data['attack_type'].isin([i+1])]
        x2 = data1.sample(n = 1000, random_state = i + 43, axis = 0)   
        data_selected = pd.concat([x1, x2], ignore_index=True)  
        data_list.append(data_selected) 
for i in range(len(data_list)):
    print(len(data_list[i]))  




2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000


In [17]:
data_list[9].tail()

,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,http.content_length,http.response,tcp.ack,tcp.ack_raw,tcp.checksum,tcp.connection.fin,...,mqtt.conflag.cleansess,mqtt.conflags,mqtt.hdrflags,mqtt.len,mqtt.msgtype,mqtt.proto_len,mqtt.topic_len,mqtt.ver,Attack_label,attack_type
1995,0,0,0,0,0,0,150,1934832499,19721,0,...,0,0,0,0,0,0,0,0,1,10
1996,0,0,0,0,0,0,0,0,24507,0,...,0,0,0,0,0,0,0,0,1,10
1997,0,0,0,0,0,0,1,3703765364,46928,0,...,0,0,0,0,0,0,0,0,1,10
1998,0,0,0,0,0,0,1,208620284,44688,0,...,0,0,0,0,0,0,0,0,1,10
1999,0,0,0,0,0,0,1,3557485597,42863,0,...,0,0,0,0,0,0,0,0,1,10


In [18]:
rs = 42 # set the random state

In [19]:
from sklearn.svm import SVC
clfs = [] # This is going to contain 14 different classifiers
n_samples = []
x_train_list = []
y_train_list = []
x_test_list = []
y_test_list = []
y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
for i in tqdm(range(len(input_file))): # reading the data
    data0 = data_list[i]
    y[i][data0.attack_type == 0] = 0
    n_samples.append(len(y[i])) # the number of this node
    x = data_list[i]
    x = x.drop(columns='attack_type')
    x = x.drop(columns='Attack_label')
    x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = rs)
    x_train_list.append(x_train)
    y_train_list.append(y_train)
    x_test_list.append(x_test)
    y_test_list.append(y_test)

x_train_full = pd.concat(x_train_list, ignore_index=True)
scaler = scale.StandardScaler().fit(x_train_full)

for i in range(len(input_file)):
    x_train_list[i] = scaler.transform(x_train_list[i])

for i in tqdm(range(len(input_file))):
    #classifier = SVC(kernel = 'rbf', random_state = 41, gamma='scale',max_iter=-1, probability=True)
    # classifier = SVC(kernel = 'rbf', random_state = rs, gamma='scale', max_iter=-1, probability=True)
    # classifier.fit(data_list[i], y[i])
    classifier = LogisticRegression(max_iter=10000)
    classifier.fit(x_train_list[i], y_train_list[i])
    clfs.append(classifier)

total_n_samples = np.sum(n_samples)
print('total number is ',total_n_samples)
p_nodes =  np.array(n_samples) / total_n_samples
print('The proportion of the nodes are:', p_nodes)



total number is  24000
The proportion of the nodes are: [0.08333333 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333
 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333]


In [20]:
for i in range(len(input_file)):
    x_test_list[i] = scaler.transform(x_test_list[i])

In [21]:
x_test = np.vstack(x_test_list)
y_test = np.hstack(y_test_list)
x_train = np.vstack(x_train_list)
y_train = np.hstack(y_train_list)

In [22]:
# GMM
from sklearn.mixture import GaussianMixture
gms = []
K = 15 # The number of the components of the GMM, one could change it by analyzing the graph
for i in tqdm(range(len(input_file))):
    x = x_train_list[i]
    gm = GaussianMixture(n_components = K).fit(x)  
    gms.append(gm)

In [23]:

prediction = pred(x_test, clfs, p_nodes, gms)

In [24]:

correct = prediction == y_test
accuracy = np.mean(correct)
print('Overall accuracy is:', accuracy)

Overall accuracy is: 0.7555


In [25]:
accs = []
for i in range(len(input_file)+1):
    if len(correct[y_test==i]) == 0:
        accs.append(0)
    else:
        accs.append(np.mean(correct[y_test==i]))
print('Accuracies:', accs)
recalls = []
for i in range(len(input_file)+1):
    if len(correct[prediction==i]) == 0:
        recalls.append(0)
    else:
        recalls.append(np.mean(correct[prediction==i]))
print('Recalls:', recalls)

Accuracies: [0.9818417639429312, 0.7119341563786008, 0.31275720164609055, 0.9958847736625515, 0.7777777777777778, 1.0, 0.12757201646090535, 0.7695473251028807, 0.2962962962962963, 0.13991769547325103, 0.14814814814814814, 0.7777777777777778, 0.13580246913580246]
Recalls: [0.7463643086024155, 0.6628352490421456, 0.5467625899280576, 1.0, 0.7297297297297297, 0.8408304498269896, 0.7045454545454546, 0.6678571428571428, 0.8, 0.7391304347826086, 0.6545454545454545, 0.9947368421052631, 0.6875]


Repeat the test:

Binary = LR:

In [31]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1 
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  
    #Training

    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x = x.drop(columns=['attack_type' , 'Attack_label'])
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)

    for i in range(len(input_file)):
        x_train_list[i] = scaler.transform(x_train_list[i])

    for i in range(len(input_file)):
        classifier = LogisticRegression(random_state=epoch)
        #classifier = KNeighborsClassifier()
        classifier.fit(x_train_list[i], y_train_list[i])
        clfs.append(classifier)

    total_n_samples = np.sum(n_samples)
    p_nodes =  np.array(n_samples) / total_n_samples


    # Test:
    for i in range(len(input_file)):
        x_test_list[i] = scaler.transform(x_test_list[i])

    x_test = np.vstack(x_test_list)
    y_test = np.hstack(y_test_list)
    x_train = np.vstack(x_train_list)
    y_train = np.hstack(y_train_list)

    from sklearn.mixture import GaussianMixture
    gms = []
    K = 15 # The number of the components of the GMM, one could change it by analyzing the graph
    for i in range(len(input_file)):
        x = x_train_list[i]
        gm = GaussianMixture(n_components = K).fit(x)  
        gms.append(gm)

    prediction = pred(x_test, clfs, p_nodes, gms)
    correct = prediction == y_test

    accs = []
    for i in range(len(input_file)+1):
        if len(correct[y_test==i]) == 0:
            accs.append(0)
        else:
            accs.append(np.mean(correct[y_test==i]))
    
    recall[epoch,:] = accs

    recalls = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            recalls.append(0)
        else:
            recalls.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = recalls

for i in range(len(input_file)+1):
      print(i,'Precision mean=',np.mean(precision[:,i]))
      print(i,'Precision std=',np.std(precision[:,i]))

for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))


0 Precision mean= 0.7388841021191862
0 Precision std= 0.03215557720512144
1 Precision mean= 0.6875197958627732
1 Precision std= 0.1738185152489118
2 Precision mean= 0.6154510861252681
2 Precision std= 0.19700052128947293
3 Precision mean= 1.0
3 Precision std= 0.0
4 Precision mean= 0.7236161533757619
4 Precision std= 0.05615195334130809
5 Precision mean= 1.0
5 Precision std= 0.0
6 Precision mean= 0.5882672948560528
6 Precision std= 0.18970598208380463
7 Precision mean= 0.6808462823304577
7 Precision std= 0.041437735026658096
8 Precision mean= 0.5558465425353457
8 Precision std= 0.30079230211237074
9 Precision mean= 0.5964192773525371
9 Precision std= 0.3243603233482481
10 Precision mean= 0.6909129793233917
10 Precision std= 0.10758606039130725
11 Precision mean= 0.9367959611563638
11 Precision std= 0.0666715971965308
12 Precision mean= 0.5901058674906204
12 Precision std= 0.2788287982297519
Recall mean of  0  = 0.8381050836996433
Recall std of  0  = 0.16535749756296866
Recall mean of  

In [35]:
for i in range(len(input_file)+1):
      print(round(100 * np.mean(precision[:,i]),2),'\% (',round(np.std(precision[:,i]),2),') &' )


73.89 \% ( 0.03 ) &
68.75 \% ( 0.17 ) &
61.55 \% ( 0.2 ) &
100.0 \% ( 0.0 ) &
72.36 \% ( 0.06 ) &
100.0 \% ( 0.0 ) &
58.83 \% ( 0.19 ) &
68.08 \% ( 0.04 ) &
55.58 \% ( 0.3 ) &
59.64 \% ( 0.32 ) &
69.09 \% ( 0.11 ) &
93.68 \% ( 0.07 ) &
59.01 \% ( 0.28 ) &


binary = KNN

In [36]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1 
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  
    #Training

    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x = x.drop(columns=['attack_type' , 'Attack_label'])
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)

    for i in range(len(input_file)):
        x_train_list[i] = scaler.transform(x_train_list[i])

    for i in range(len(input_file)):
        #classifier = LogisticRegression(random_state=epoch)
        classifier = KNeighborsClassifier()
        classifier.fit(x_train_list[i], y_train_list[i])
        clfs.append(classifier)

    total_n_samples = np.sum(n_samples)
    p_nodes =  np.array(n_samples) / total_n_samples


    # Test:
    for i in range(len(input_file)):
        x_test_list[i] = scaler.transform(x_test_list[i])

    x_test = np.vstack(x_test_list)
    y_test = np.hstack(y_test_list)
    x_train = np.vstack(x_train_list)
    y_train = np.hstack(y_train_list)

    from sklearn.mixture import GaussianMixture
    gms = []
    K = 15 # The number of the components of the GMM, one could change it by analyzing the graph
    for i in range(len(input_file)):
        x = x_train_list[i]
        gm = GaussianMixture(n_components = K).fit(x)  
        gms.append(gm)

    prediction = pred(x_test, clfs, p_nodes, gms)
    correct = prediction == y_test

    accs = []
    for i in range(len(input_file)+1):
        if len(correct[y_test==i]) == 0:
            accs.append(0)
        else:
            accs.append(np.mean(correct[y_test==i]))
    
    recall[epoch,:] = accs

    recalls = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            recalls.append(0)
        else:
            recalls.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = recalls

for i in range(len(input_file)+1):
      print(i,'Precision mean=',np.mean(precision[:,i]))
      print(i,'Precision std=',np.std(precision[:,i]))

for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))

print('Latex Precisions')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(precision[:,i]),2),'\% (',round(np.std(precision[:,i]),2),') &' )


print('Latex Recalls')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(recall[:,i]),2),'\% (',round(np.std(recall[:,i]),2),') &' )


0 Precision mean= 0.741507689496051
0 Precision std= 0.01816138052017399
1 Precision mean= 0.7388803438632314
1 Precision std= 0.17270496227344936
2 Precision mean= 0.5920844582584969
2 Precision std= 0.1892334181212948
3 Precision mean= 1.0
3 Precision std= 0.0
4 Precision mean= 0.7891732926877979
4 Precision std= 0.08641365967154921
5 Precision mean= 1.0
5 Precision std= 0.0
6 Precision mean= 0.6267264582534711
6 Precision std= 0.14466020863115606
7 Precision mean= 0.652270671324659
7 Precision std= 0.05770101050754602
8 Precision mean= 0.5725541827806991
8 Precision std= 0.21248709217219927
9 Precision mean= 0.34411675389821356
9 Precision std= 0.2977935151014835
10 Precision mean= 0.6572695034045392
10 Precision std= 0.18638565685021172
11 Precision mean= 0.962273680960766
11 Precision std= 0.0338869107395621
12 Precision mean= 0.555114385200237
12 Precision std= 0.32179480773472635
Recall mean of  0  = 0.8704032836227213
Recall std of  0  = 0.08442241277211392
Recall mean of  1  

SVM Binary

In [42]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1 
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  
    #Training

    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x = x.drop(columns=['attack_type' , 'Attack_label'])
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)

    for i in range(len(input_file)):
        x_train_list[i] = scaler.transform(x_train_list[i])

    for i in range(len(input_file)):
        classifier = SVC(kernel = 'rbf', random_state = 41, gamma='scale',max_iter=-1, probability=True)
        #classifier = KNeighborsClassifier()
        classifier.fit(x_train_list[i], y_train_list[i])
        clfs.append(classifier)

    total_n_samples = np.sum(n_samples)
    p_nodes =  np.array(n_samples) / total_n_samples


    # Test:
    for i in range(len(input_file)):
        x_test_list[i] = scaler.transform(x_test_list[i])

    x_test = np.vstack(x_test_list)
    y_test = np.hstack(y_test_list)
    x_train = np.vstack(x_train_list)
    y_train = np.hstack(y_train_list)

    from sklearn.mixture import GaussianMixture
    gms = []
    K = 15 # The number of the components of the GMM, one could change it by analyzing the graph
    for i in range(len(input_file)):
        x = x_train_list[i]
        gm = GaussianMixture(n_components = K).fit(x)  
        gms.append(gm)

    prediction = pred(x_test, clfs, p_nodes, gms)
    correct = prediction == y_test

    accs = []
    for i in range(len(input_file)+1):
        if len(correct[y_test==i]) == 0:
            accs.append(0)
        else:
            accs.append(np.mean(correct[y_test==i]))
    
    recall[epoch,:] = accs

    recalls = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            recalls.append(0)
        else:
            recalls.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = recalls

for i in range(len(input_file)+1):
      print(i,'Precision mean=',np.mean(precision[:,i]))
      print(i,'Precision std=',np.std(precision[:,i]))

for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))




0 Precision mean= 0.7447387161457149
0 Precision std= 0.029660753238044495
1 Precision mean= 0.7239339189524244
1 Precision std= 0.21224032860668135
2 Precision mean= 0.6981789688330158
2 Precision std= 0.24329846037681863
3 Precision mean= 1.0
3 Precision std= 0.0
4 Precision mean= 0.7565899616901943
4 Precision std= 0.07736294100474271
5 Precision mean= 0.9654972195737137
5 Precision std= 0.06926349511885513
6 Precision mean= 0.6662561793470453
6 Precision std= 0.22064104079370042
7 Precision mean= 0.6647609950545628
7 Precision std= 0.053573270613164026
8 Precision mean= 0.5497520141316414
8 Precision std= 0.2433951675206493
9 Precision mean= 0.5622527591601025
9 Precision std= 0.3440716143636287
10 Precision mean= 0.706724704888271
10 Precision std= 0.0998603411824382
11 Precision mean= 0.9220472278300467
11 Precision std= 0.07121234416238494
12 Precision mean= 0.6829345996690833
12 Precision std= 0.3280296057640471
Recall mean of  0  = 0.8260893247396301
Recall std of  0  = 0.163

In [43]:
print('Latex Precisions')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(precision[:,i]),2),'\% (',round(np.std(precision[:,i]),2),') &' )


print('Latex Recalls')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(recall[:,i]),2),'\% (',round(np.std(recall[:,i]),2),') &' )

Latex Precisions
74.47 \% ( 0.03 ) &
72.39 \% ( 0.21 ) &
69.82 \% ( 0.24 ) &
100.0 \% ( 0.0 ) &
75.66 \% ( 0.08 ) &
96.55 \% ( 0.07 ) &
66.63 \% ( 0.22 ) &
66.48 \% ( 0.05 ) &
54.98 \% ( 0.24 ) &
56.23 \% ( 0.34 ) &
70.67 \% ( 0.1 ) &
92.2 \% ( 0.07 ) &
68.29 \% ( 0.33 ) &
Latex Recalls
82.61 \% ( 0.16 ) &
56.68 \% ( 0.2 ) &
26.49 \% ( 0.09 ) &
100.0 \% ( 0.0 ) &
76.88 \% ( 0.09 ) &
99.98 \% ( 0.0 ) &
12.67 \% ( 0.09 ) &
75.7 \% ( 0.1 ) &
29.2 \% ( 0.2 ) &
24.0 \% ( 0.21 ) &
13.93 \% ( 0.02 ) &
78.02 \% ( 0.03 ) &
16.53 \% ( 0.13 ) &


RF binary

In [45]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1 
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  
    #Training

    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x = x.drop(columns=['attack_type' , 'Attack_label'])
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)

    for i in range(len(input_file)):
        x_train_list[i] = scaler.transform(x_train_list[i])

    for i in range(len(input_file)):
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = rs)
        classifier.fit(x_train_list[i], y_train_list[i])
        clfs.append(classifier)

    total_n_samples = np.sum(n_samples)
    p_nodes =  np.array(n_samples) / total_n_samples


    # Test:
    for i in range(len(input_file)):
        x_test_list[i] = scaler.transform(x_test_list[i])

    x_test = np.vstack(x_test_list)
    y_test = np.hstack(y_test_list)
    x_train = np.vstack(x_train_list)
    y_train = np.hstack(y_train_list)

    from sklearn.mixture import GaussianMixture
    gms = []
    K = 15 # The number of the components of the GMM, one could change it by analyzing the graph
    for i in range(len(input_file)):
        x = x_train_list[i]
        gm = GaussianMixture(n_components = K).fit(x)  
        gms.append(gm)

    prediction = pred(x_test, clfs, p_nodes, gms)
    correct = prediction == y_test

    accs = []
    for i in range(len(input_file)+1):
        if len(correct[y_test==i]) == 0:
            accs.append(0)
        else:
            accs.append(np.mean(correct[y_test==i]))
    
    recall[epoch,:] = accs

    recalls = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            recalls.append(0)
        else:
            recalls.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = recalls

for i in range(len(input_file)+1):
      print(i,'Precision mean=',np.mean(precision[:,i]))
      print(i,'Precision std=',np.std(precision[:,i]))

for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))




0 Precision mean= 0.8006090080302325
0 Precision std= 0.04380101633269852
1 Precision mean= 0.778464254853118
1 Precision std= 0.1694008158871531
2 Precision mean= 0.5282633338775911
2 Precision std= 0.18311822541573566
3 Precision mean= 1.0
3 Precision std= 0.0
4 Precision mean= 0.9860622662715086
4 Precision std= 0.028052077490259553
5 Precision mean= 1.0
5 Precision std= 0.0
6 Precision mean= 0.6342802369686409
6 Precision std= 0.19945133766872472
7 Precision mean= 0.7226401178321804
7 Precision std= 0.07771506358806067
8 Precision mean= 0.6185142578486207
8 Precision std= 0.23131752682581408
9 Precision mean= 0.4779468849331163
9 Precision std= 0.2370162026242056
10 Precision mean= 0.6951328855966932
10 Precision std= 0.09012678963310154
11 Precision mean= 0.9508998894343581
11 Precision std= 0.04766869432546471
12 Precision mean= 0.439416478398252
12 Precision std= 0.21582984116456264
Recall mean of  0  = 0.9727090870428023
Recall std of  0  = 0.02045218508577791
Recall mean of  

In [46]:
print('Latex Precisions')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(precision[:,i]),2),'\% (',round(np.std(precision[:,i]),2),') &' )


print('Latex Recalls')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(recall[:,i]),2),'\% (',round(np.std(recall[:,i]),2),') &' )

Latex Precisions
80.06 \% ( 0.04 ) &
77.85 \% ( 0.17 ) &
52.83 \% ( 0.18 ) &
100.0 \% ( 0.0 ) &
98.61 \% ( 0.03 ) &
100.0 \% ( 0.0 ) &
63.43 \% ( 0.2 ) &
72.26 \% ( 0.08 ) &
61.85 \% ( 0.23 ) &
47.79 \% ( 0.24 ) &
69.51 \% ( 0.09 ) &
95.09 \% ( 0.05 ) &
43.94 \% ( 0.22 ) &
Latex Recalls
97.27 \% ( 0.02 ) &
62.54 \% ( 0.17 ) &
29.99 \% ( 0.12 ) &
99.96 \% ( 0.0 ) &
83.0 \% ( 0.12 ) &
99.9 \% ( 0.0 ) &
15.26 \% ( 0.08 ) &
87.22 \% ( 0.17 ) &
35.95 \% ( 0.21 ) &
35.87 \% ( 0.19 ) &
14.01 \% ( 0.03 ) &
78.52 \% ( 0.04 ) &
23.96 \% ( 0.15 ) &


Full:

LR

In [39]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  
    #Training


    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x = x.drop(columns=['attack_type','Attack_label'])
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch+40)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    y_train_full = np.hstack(y_train_list)
    x_test_full = pd.concat(x_test_list, ignore_index=True)
    #y_test_full = pd.concat(y_test_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)
    x_train_full = scaler.transform(x_train_full)
    classifier = LogisticRegression(random_state = epoch)
    classifier.fit(x_train_full, y_train_full)

    x_test_full = scaler.transform(x_test_full)

    #x_test = np.vstack(x_test_list)
    y_test_full = np.hstack(y_test_list)
   # x_train = np.vstack(x_train_list)
    


    prediction = classifier.predict(x_test_full)
    correct = prediction == y_test_full

    rec = []
    for i in range(len(input_file)+1):
        if len(correct[y_test_full==i]) == 0:
            rec.append(0)
        else:
            rec.append(np.mean(correct[y_test_full==i]))
    
    recall[epoch,:] = rec
      

    pr = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            pr.append(0)
        else:
            pr.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = pr
    
for i in range(len(input_file)+1):
      print(i,'Precision mean=',np.mean(precision[:,i]))
      print(i,'Precision std=',np.std(precision[:,i]))

for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))
      
print('Latex Precisions')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(precision[:,i]),2),'\% (',round(np.std(precision[:,i]),2),') &' )


print('Latex Recalls')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(recall[:,i]),2),'\% (',round(np.std(recall[:,i]),2),') &' )


/Users/irohayachiyo/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/irohayachiyo/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo


0 Precision mean= 0.6972837585359323
0 Precision std= 0.018303512465238474
1 Precision mean= 0.4038418216092398
1 Precision std= 0.03937783178100957
2 Precision mean= 0.527915416028818
2 Precision std= 0.04127416591604016
3 Precision mean= 0.9986146263616664
3 Precision std= 0.002886313340697261
4 Precision mean= 0.7418605439876964
4 Precision std= 0.06688044980148687
5 Precision mean= 0.9993941852425012
5 Precision std= 0.001442356923587901
6 Precision mean= 0.8206673433801329
6 Precision std= 0.14557647066506124
7 Precision mean= 0.6777563878212962
7 Precision std= 0.06208623882638545
8 Precision mean= 0.3309501884819273
8 Precision std= 0.05465581264530748
9 Precision mean= 0.49680769876716424
9 Precision std= 0.0770678917169437
10 Precision mean= 0.6444430145872899
10 Precision std= 0.1020292679184349
11 Precision mean= 0.8785379113370929
11 Precision std= 0.01608047448712708
12 Precision mean= 0.0
12 Precision std= 0.0
Recall mean of  0  = 0.9908181767574199
Recall std of  0  = 0

/Users/irohayachiyo/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


KNN Full

In [41]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  
    #Training


    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x = x.drop(columns=['attack_type','Attack_label'])
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch+40)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    y_train_full = np.hstack(y_train_list)
    x_test_full = pd.concat(x_test_list, ignore_index=True)
    #y_test_full = pd.concat(y_test_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)
    x_train_full = scaler.transform(x_train_full)
    classifier = KNeighborsClassifier()
    classifier.fit(x_train_full, y_train_full)

    x_test_full = scaler.transform(x_test_full)

    #x_test = np.vstack(x_test_list)
    y_test_full = np.hstack(y_test_list)
   # x_train = np.vstack(x_train_list)
    


    prediction = classifier.predict(x_test_full)
    correct = prediction == y_test_full

    rec = []
    for i in range(len(input_file)+1):
        if len(correct[y_test_full==i]) == 0:
            rec.append(0)
        else:
            rec.append(np.mean(correct[y_test_full==i]))
    
    recall[epoch,:] = rec
      

    pr = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            pr.append(0)
        else:
            pr.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = pr
    
for i in range(len(input_file)+1):
      print(i,'Precision mean=',np.mean(precision[:,i]))
      print(i,'Precision std=',np.std(precision[:,i]))

for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))
      
print('Latex Precisions')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(precision[:,i]),2),'\% (',round(np.std(precision[:,i]),2),') &' )


print('Latex Recalls')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(recall[:,i]),2),'\% (',round(np.std(recall[:,i]),2),') &' )



0 Precision mean= 0.7480887457555241
0 Precision std= 0.016593364965375768
1 Precision mean= 0.5419835949755898
1 Precision std= 0.028222316490708946
2 Precision mean= 0.48929620907957927
2 Precision std= 0.03691055437913015
3 Precision mean= 0.9962746197645146
3 Precision std= 0.0035993217922279234
4 Precision mean= 0.7095183709391046
4 Precision std= 0.02224567467586668
5 Precision mean= 0.9980016885928794
5 Precision std= 0.0032779936852438754
6 Precision mean= 0.33120715558394476
6 Precision std= 0.07247355892328458
7 Precision mean= 0.7035021999864176
7 Precision std= 0.027260892697049437
8 Precision mean= 0.687061573274929
8 Precision std= 0.03701352708301233
9 Precision mean= 0.4018288689149343
9 Precision std= 0.05541492568783841
10 Precision mean= 0.38269939622721905
10 Precision std= 0.04870929614910959
11 Precision mean= 0.8470160659294919
11 Precision std= 0.022105486316875343
12 Precision mean= 0.20763897919367
12 Precision std= 0.05495894102530598
Recall mean of  0  = 0.

RF Full

In [48]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  
    #Training


    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x = x.drop(columns=['attack_type','Attack_label'])
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch+40)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    y_train_full = np.hstack(y_train_list)
    x_test_full = pd.concat(x_test_list, ignore_index=True)
    #y_test_full = pd.concat(y_test_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)
    x_train_full = scaler.transform(x_train_full)
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = rs + i)
    classifier.fit(x_train_full, y_train_full)

    x_test_full = scaler.transform(x_test_full)

    #x_test = np.vstack(x_test_list)
    y_test_full = np.hstack(y_test_list)
   # x_train = np.vstack(x_train_list)
    


    prediction = classifier.predict(x_test_full)
    correct = prediction == y_test_full

    rec = []
    for i in range(len(input_file)+1):
        if len(correct[y_test_full==i]) == 0:
            rec.append(0)
        else:
            rec.append(np.mean(correct[y_test_full==i]))
    
    recall[epoch,:] = rec
      

    pr = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            pr.append(0)
        else:
            pr.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = pr
    
for i in range(len(input_file)+1):
      print(i,'Precision mean=',np.mean(precision[:,i]))
      print(i,'Precision std=',np.std(precision[:,i]))

for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))
      
print('Latex Precisions')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(precision[:,i]),2),'\% (',round(np.std(precision[:,i]),2),') &' )


print('Latex Recalls')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(recall[:,i]),2),'\% (',round(np.std(recall[:,i]),2),') &' )



0 Precision mean= 0.90589853888143
0 Precision std= 0.006580613078238353
1 Precision mean= 0.9366169078043483
1 Precision std= 0.014485752411366626
2 Precision mean= 0.6415434786173213
2 Precision std= 0.02361746883293334
3 Precision mean= 0.9995997694672131
3 Precision std= 0.0012010757582163777
4 Precision mean= 0.9997863247863247
4 Precision std= 0.0009313886631497133
5 Precision mean= 0.9985836138618666
5 Precision std= 0.0030048836765764414
6 Precision mean= 0.8113134230011194
6 Precision std= 0.02879803513308709
7 Precision mean= 0.8307061444642816
7 Precision std= 0.021755638790884855
8 Precision mean= 0.8064086597578095
8 Precision std= 0.03755852021785365
9 Precision mean= 0.7592343586622707
9 Precision std= 0.034898868632500524
10 Precision mean= 0.6729387806635958
10 Precision std= 0.025266515705357595
11 Precision mean= 0.9636122397297683
11 Precision std= 0.010653571517084418
12 Precision mean= 0.8107260470177945
12 Precision std= 0.04936101341480204
Recall mean of  0  = 

SVM Full

In [49]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  
    #Training


    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x = x.drop(columns=['attack_type','Attack_label'])
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch+40)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    y_train_full = np.hstack(y_train_list)
    x_test_full = pd.concat(x_test_list, ignore_index=True)
    #y_test_full = pd.concat(y_test_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)
    x_train_full = scaler.transform(x_train_full)
    classifier = SVC(kernel = 'rbf', random_state = 41, gamma='scale',max_iter=-1, probability=True)
    classifier.fit(x_train_full, y_train_full)

    x_test_full = scaler.transform(x_test_full)

    #x_test = np.vstack(x_test_list)
    y_test_full = np.hstack(y_test_list)
   # x_train = np.vstack(x_train_list)
    


    prediction = classifier.predict(x_test_full)
    correct = prediction == y_test_full

    rec = []
    for i in range(len(input_file)+1):
        if len(correct[y_test_full==i]) == 0:
            rec.append(0)
        else:
            rec.append(np.mean(correct[y_test_full==i]))
    
    recall[epoch,:] = rec
      

    pr = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            pr.append(0)
        else:
            pr.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = pr
    
for i in range(len(input_file)+1):
      print(i,'Precision mean=',np.mean(precision[:,i]))
      print(i,'Precision std=',np.std(precision[:,i]))

for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))
      
print('Latex Precisions')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(precision[:,i]),2),'\% (',round(np.std(precision[:,i]),2),') &' )


print('Latex Recalls')
for i in range(len(input_file)+1):
      print(round(100 * np.mean(recall[:,i]),2),'\% (',round(np.std(recall[:,i]),2),') &' )



0 Precision mean= 0.7018560977899171
0 Precision std= 0.01823249266520187
1 Precision mean= 0.4211066053685132
1 Precision std= 0.0765853174386387
2 Precision mean= 0.4711404864775954
2 Precision std= 0.04005645546023005
3 Precision mean= 0.9966277564913211
3 Precision std= 0.003125778608181801
4 Precision mean= 0.9524019413331141
4 Precision std= 0.10602799176390985
5 Precision mean= 0.8435448069392588
5 Precision std= 0.054839739133382644
6 Precision mean= 0.9898793363499246
6 Precision std= 0.024262373302972276
7 Precision mean= 0.5981090682805664
7 Precision std= 0.03677171817651272
8 Precision mean= 0.4164394749442808
8 Precision std= 0.10324497402064176
9 Precision mean= 0.8020319943349445
9 Precision std= 0.1013041515008211
10 Precision mean= 0.7064000182877034
10 Precision std= 0.06939506059932997
11 Precision mean= 0.8770294682065561
11 Precision std= 0.015897778270731723
12 Precision mean= 0.0
12 Precision std= 0.0
Recall mean of  0  = 0.9866006485198149
Recall std of  0  = 